In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.providers.aer import QasmSimulator

import numpy as np
from bitstring import BitArray

# the following function builds a gate (oracle) that transforms |x>|y> to |x>|y - cut(x)>
def EnergyOracle(GraphLaplacian: list[list[int]], digits: int):

    # setting up the circuit
    QRegX = QuantumRegister(len(GraphLaplacian),"x")
    QRegY = QuantumRegister(digits,"y")

    QC = QuantumCircuit(QRegX,QRegY)

    # QFT
    for i in range(digits):
        QC.h(QRegY[i])

        theta = np.pi/2
        for j in range(i + 1, digits):
            QC.cp(theta, QRegY[j], QRegY[i])
            theta /= 2

    for i in range(digits >> 1):
        QC.swap(QRegY[i],QRegY[- (i + 1)])

    # Phasers
    for i in range(len(GraphLaplacian)):
        theta = - GraphLaplacian[i][i] * np.pi
        for j in range(digits):
            QC.cp(theta, QRegX[i], QRegY[j])
            theta /= 2

        for j in range(i + 1, len(GraphLaplacian[i])):
            if GraphLaplacian[i][j] != 0:
                theta = - GraphLaplacian[i][j] * np.pi * 2
                for k in range(digits):
                    QC.mcp(theta, [QRegX[i], QRegX[j]], QRegY[k])
                    theta /= 2

    # Inverse QFT
    for i in range(digits >> 1):
        QC.swap(QRegY[i],QRegY[- (i + 1)])

    for i in range(digits - 1, - 1, - 1):
        QC.h(QRegY[i])

        theta = - np.pi/2
        for j in range(i - 1, - 1, - 1):
            QC.cp(theta, QRegY[j], QRegY[i])
            theta /= 2

    return QC.to_gate()



# the following function builds the G(alpha, beta) gates for the fixed point search
def GroverBangBang(GraphLaplacian: list[list[int]], digits: int, alpha: float, beta: float):
    
    U_f = EnergyOracle(GraphLaplacian, digits)
    U_f_inv = U_f.inverse()

    QRegX = QuantumRegister(len(GraphLaplacian))
    QRegY = QuantumRegister(digits)

    GroverBangBang = QuantumCircuit(QRegX,QRegY)

    # S_t (beta)
    GroverBangBang.append(U_f, QRegX[:] + QRegY[:])
    GroverBangBang.p(beta, QRegY[0])
    GroverBangBang.append(U_f_inv, QRegX[:] + QRegY[:])

    # S_s (alpha)
    GroverBangBang.h(QRegX)
    GroverBangBang.x(QRegX)
    GroverBangBang.mcp(- alpha, QRegX[:-1], QRegX[-1])
    GroverBangBang.x(QRegX)
    GroverBangBang.h(QRegX)

    return GroverBangBang.to_gate()



# creates the S_L circuit
def GroverFixedPointGate(GraphLaplacian: list[list[int]], digits: int, l: int, delta: float):

    gamma = np.sqrt(1 - pow(1 / np.cosh(np.arccosh(1 / delta) / (2 * l + 1)), 2))

    alpha = []

    for j in range(1, l + 1):        
        alpha.append(2 * np.arctan(1 / (gamma * np.tan(2 * np.pi * j / (2 * l + 1)))))

    QRegX = QuantumRegister(len(GraphLaplacian), "x")
    QRegY = QuantumRegister(digits, "y")

    QC = QuantumCircuit(QRegX,QRegY)

    QC.h(QRegX)

    for j in range(l):
        QC.append(GroverBangBang(GraphLaplacian, digits, alpha[j], - alpha[l - 1 - j]), QRegX[:] + QRegY[:])

    # add one more energy oracle to see the cut values
    QC.append(EnergyOracle(GraphLaplacian, digits), QRegX[:] + QRegY[:])

    return QC.to_gate()



# builds the circuit for the new QAOA
def GroverQAOACircuit(GraphLaplacian: list[list[int]], digits: int, y: int, delta: float, l: int, bangs: list[list[int]], ShotNumber: int):

    GroverMixer = GroverFixedPointGate(GraphLaplacian, digits, l, delta)
    GroverMixer.label = "GroverMixer"

    InverseGroverMixer = GroverMixer.inverse()
    InverseGroverMixer.label = "InverseGroverMixer"

    # one quantum/classical register pair for each vertex
    QRegX = QuantumRegister(len(GraphLaplacian), "x")
    ClRegX = ClassicalRegister(len(GraphLaplacian), "cl-x")

    # one quantum/classical register pair for each digit
    QRegY = QuantumRegister(digits, "y")
    ClRegY = ClassicalRegister(digits, "cl-y")

    QC = QuantumCircuit(QRegX,QRegY,ClRegX,ClRegY)

    for i in range(len(QRegX)):
        QC.initialize([1, 0], QRegX[i])

    for i in range(len(QRegY)):
        QC.initialize([1 - (y >> (digits - i - 1))%2, (y >> (digits - i - 1))%2], QRegY[i])

    QC.append(GroverMixer, QRegX[:] + QRegY[:])

    # the bangs / p = len(bangs) / going backward per convention
    for k in range(len(bangs) - 1, - 1, - 1):

        #this implements the phase-bang (without totterization!!!)
        for i in range(digits):
            QC.p(bangs[k][1] * (1 << (digits - 1 - i)), QRegY[i])

        # this is the Grover-bang
        QC.append(InverseGroverMixer, QRegX[:] + QRegY[:])
        QC.x(QRegX)
        QC.mcp(bangs[k][0], QRegX[:-1], QRegX[-1])
        QC.x(QRegX)
        QC.append(GroverMixer, QRegX[:] + QRegY[:])

    # measure
    QC.measure(QRegX, ClRegX)
    QC.measure(QRegY, ClRegY)

    print()
    print(QC)

    # simulate
    simulator = QasmSimulator()
    compiled_QC = transpile(QC, simulator)
    job = simulator.run(compiled_QC, shots=ShotNumber)
    result = job.result()

    return result.get_counts(compiled_QC)



# TEST

#GraphLaplacian = [[3, - 1, - 1, - 1], [- 1, 1, 0, 0], [- 1, 0, 1, 0], [- 1, 0, 0, 1]]
GraphLaplacian = [[4, - 1, - 1, - 1, - 1], [- 1, 3, - 1, -1, 0], [- 1, - 1, 3, - 1, 0], [- 1, - 1, - 1, 4, - 1], [ 1, 0, 0, - 1, 2]]
digits = 4
y = 1
P_L = 0.9
ShotNumber = 10

# ratio of good states to all states
RootLambda = 1/2

# from the input we compute the following
delta = np.sqrt(1 - P_L)
l = int(np.ceil(np.arccosh(1 / delta) / np.arccosh(1 / np.sqrt(1 - RootLambda * RootLambda)))) >> 1




# empty bangs list just makes it work like a Grover search circuit
bangs = []

# results

counts = GroverQAOACircuit(GraphLaplacian, digits, y, delta, l, bangs, ShotNumber)

adjusted_counts = []

for s in counts:
    adjusted_counts.append([s[::-1], 100 * counts[s] / ShotNumber])

adjusted_counts.sort()

print()
print("graph laplacian")
print("---------------")
for x in GraphLaplacian:
    print(x)
print()
print("energy threshold (y) =",y)
print("query complexity (l) =",l)
print("square root of lambda =",RootLambda)
print("P_L =",P_L * 100,"%")
print()
prob = 0
for s in adjusted_counts:
    b = s[0][-digits:]
    if b[0] == "1":
        prob += s[1]
    cut = BitArray(bin=b).int
    cut = y - cut
    print("configuration =",s[0][:len(GraphLaplacian)],"\tcut =",cut,"\tfrequency =",s[1],"%")
print()
print("Probability of success =",prob,"%")
print()


        ┌─────────────────┐┌──────────────┐┌─┐                        
   x_0: ┤ Initialize(1,0) ├┤0             ├┤M├────────────────────────
        ├─────────────────┤│              │└╥┘┌─┐                     
   x_1: ┤ Initialize(1,0) ├┤1             ├─╫─┤M├─────────────────────
        ├─────────────────┤│              │ ║ └╥┘┌─┐                  
   x_2: ┤ Initialize(1,0) ├┤2             ├─╫──╫─┤M├──────────────────
        ├─────────────────┤│              │ ║  ║ └╥┘┌─┐               
   x_3: ┤ Initialize(1,0) ├┤3             ├─╫──╫──╫─┤M├───────────────
        ├─────────────────┤│              │ ║  ║  ║ └╥┘┌─┐            
   x_4: ┤ Initialize(1,0) ├┤4 GroverMixer ├─╫──╫──╫──╫─┤M├────────────
        ├─────────────────┤│              │ ║  ║  ║  ║ └╥┘┌─┐         
   y_0: ┤ Initialize(1,0) ├┤5             ├─╫──╫──╫──╫──╫─┤M├─────────
        ├─────────────────┤│              │ ║  ║  ║  ║  ║ └╥┘┌─┐      
   y_1: ┤ Initialize(1,0) ├┤6             ├─╫──╫──╫──╫──╫──╫─┤M├──────
     